In [ ]:
from functions import *
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KernelDensity
from tqdm import tqdm
import math
import cmath
from scipy.integrate import simpson, romb
from scipy.stats import iqr, norm, gaussian_kde
from scipy.special import erfc, wofz
import pickle
import os
from scipy.interpolate import RegularGridInterpolator

In [ ]:
theta_list, x_list = np.loadtxt("DataPhase28.dat", unpack=True)
T = len(theta_list)
alpha = 0.3
r2 = 0.925

L_limit = 4
n_precision = 1000
X_precision = np.linspace(-L_limit, L_limit, n_precision)
N_theta = 20

grouped_theta_list, grouped_data_dict = data_wrapper(theta_list, x_list, N_theta)


In [ ]:
n = T / N_theta
mu_max = np.log(n)
N_mu = int(mu_max**2)

print(f"n: {n}, mu_max: {mu_max}, N_mu: {N_mu}")

In [ ]:
mu_list = np.linspace(-mu_max, mu_max, N_mu)

N_shape = 100
gmin, gmax = -8, 8
g_list = np.linspace(gmin, gmax, N_shape)
h_list = g_list

kappa = 0.9
correction = True
rho_heat_map = rho_max(grouped_data_dict, mu_max, N_mu, kappa, n_precision, g_list, h_list, L_limit, N_theta, correction)

In [ ]:
# grid in y and y' (Cartesian coordinates)
ymin, ymax, N = -4.0, 4.0, 401
y  = np.linspace(ymin, ymax, N)
yp = np.linspace(ymin, ymax, N)
Y, Yp = np.meshgrid(y, yp, indexing="xy")  # X-axis: y, Y-axis: y'

R = rho_kitten(Y, Yp, r2, alpha)



g_grid = np.array(g_list)
h_grid = np.array(h_list)

y = np.linspace(ymin, ymax, N)
yp = np.linspace(ymin, ymax, N)
Y, Yp = np.meshgrid(y, yp)

G = Y - Yp
H = Y + Yp

H_clipped = np.clip(H, h_grid[0], h_grid[-1])
G_clipped = np.clip(G, g_grid[0], g_grid[-1])

points = np.stack([H_clipped.ravel(), G_clipped.ravel()], axis=-1)

real_interp = RegularGridInterpolator((h_grid, g_grid), rho_heat_map.real)
imag_interp = RegularGridInterpolator((h_grid, g_grid), rho_heat_map.imag)

real_vals = real_interp(points).reshape(Y.shape)
imag_vals = imag_interp(points).reshape(Y.shape)



vmin = min(np.nanmin(real_vals), np.nanmin(R))
vmax = max(np.nanmax(real_vals), np.nanmax(R))




fig, axes = plt.subplots(1, 2, figsize=(12, 5))

im0 = axes[0].imshow(R, origin='lower', extent=[ymin, ymax, ymin, ymax],
                     aspect='equal', cmap='viridis', vmin=vmin, vmax=vmax)
axes[0].set_xlabel(r"$y$")
axes[0].set_ylabel(r"$y'$")
axes[0].set_title(r"True $\rho(y,y')$")

im1 = axes[1].imshow(real_vals, origin='lower', extent=[y[0], y[-1], yp[0], yp[-1]],
                     aspect='equal', cmap='viridis', vmin=vmin, vmax=vmax)
axes[1].set_xlabel(r"$y$")
axes[1].set_ylabel(r"$y'$")
axes[1].set_title(r"Reconstructed $\rho(y,y')$")

fig.colorbar(im1, ax=axes.ravel().tolist())




In [ ]:
np.max(np.abs(R - (real_vals))**2)